In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import networkx as nx
import glob
import os
import json

In [13]:
G = nx.read_gml('../Proj3/mention_network.gml')
analyzer = SentimentIntensityAnalyzer()

def extract_body(msg):
    content = msg.get("message", "")
    if isinstance(content, dict):
        content = content.get("body", "")
    return content if isinstance(content, str) else ""

# Find the number of messeage for each streamer (node)
files_dir = "../Proj3/mention_network_chats/"
message_counts = {}
word_counts = {}
for node in G.nodes():
    pattern = os.path.join(files_dir, f"{node.capitalize()}_*.json")
    matches = glob.glob(pattern)
    if len(matches) == 0:
        pattern = os.path.join(files_dir, f"{node}_*.json") 
        matches = glob.glob(pattern)
    try:
        with open(matches[0], 'r', encoding='utf-8') as f:
            data = json.load(f)
    except (IndexError, FileNotFoundError, json.JSONDecodeError):
        print(f"Could not find or open file for node: {node}")
        continue

    messages = data.get("comments", [])
    texts = [extract_body(msg) for msg in messages]
    sentiments = [analyzer.polarity_scores(text) for text in texts if text]
    sentiments = [s['compound'] for s in sentiments if s]
    G.nodes[node]['sentiments'] = sentiments
    

Could not find or open file for node: jasontheween
Could not find or open file for node: riotgames
Could not find or open file for node: primevideo
Could not find or open file for node: silky
Could not find or open file for node: asmongold247
Could not find or open file for node: thunderpicktv
Could not find or open file for node: unboxholics
Could not find or open file for node: theburntpeanut_247
Could not find or open file for node: cohh
Could not find or open file for node: donerlan
Could not find or open file for node: primevision
Could not find or open file for node: caseoh247
Could not find or open file for node: ray
Could not find or open file for node: hazirb
Could not find or open file for node: lirik_247
Could not find or open file for node: camy
Could not find or open file for node: tarzaned
Could not find or open file for node: lplenglish
Could not find or open file for node: fredagainagain
Could not find or open file for node: zenrll
Could not find or open file for node: 

In [8]:
print(G.nodes(data=True))

[('caedrel', {'display_name': 'Caedrel', 'rank': 1, 'avg_sentiment': {'neg': 0.053984111568920586, 'neu': 0.8084800519855738, 'pos': 0.1375356293603888, 'compound': 0.044205764822662065}}), ('jasontheween', {'display_name': 'jasontheween', 'rank': 2}), ('zackrawrr', {'display_name': 'zackrawrr', 'rank': 3, 'avg_sentiment': {'neg': 0.08685412848681234, 'neu': 0.7625374069761747, 'pos': 0.15060786715941962, 'compound': 0.029847390269829667}}), ('stableronaldo', {'display_name': 'stableronaldo', 'rank': 4, 'avg_sentiment': {'neg': 0.03686002500656, 'neu': 0.8063836654677737, 'pos': 0.1567562940895004, 'compound': 0.0571718213111433}}), ('caseoh_', {'display_name': 'caseoh_', 'rank': 5, 'avg_sentiment': {'neg': 0.05177601574788688, 'neu': 0.8076630485774615, 'pos': 0.1405606413216478, 'compound': 0.05286626990332647}}), ('lacy', {'display_name': 'Lacy', 'rank': 6, 'avg_sentiment': {'neg': 0.057558470305656126, 'neu': 0.8349617105449073, 'pos': 0.10747963076343474, 'compound': 0.02821809918

In [12]:
# find the top 10 most positive and negative streamers
sentiment_scores = {
    node: attrs["avg_sentiment"]["compound"]
    for node, attrs in G.nodes(data=True)
    if "avg_sentiment" in attrs}
sorted_streamers = sorted(sentiment_scores.items(), key=lambda item: item[1])
top_10_negative = sorted_streamers[:10]
top_10_positive = sorted_streamers[-10:]

print("--- Top 10 Negative Streamers: ---")
for i, (streamer, score) in enumerate(top_10_negative):
    print(f"{i+1}. {streamer}: {(score*100) :.2f}")

print("\n--- Top 10 Positive Streamers: ---")
for i, (streamer, score) in enumerate(reversed(top_10_positive)):
    print(f"{i+1}. {streamer}: {(score*100):.2f}")
    

--- Top 10 Negative Streamers: ---
1. eamaddennfl: -10.03
2. tray: -2.23
3. forsen: -1.57
4. faister: -1.31
5. brucedropemoff: -1.27
6. erobb221: -1.21
7. sneakylol: -0.93
8. kurt0411: -0.68
9. ereklo_game: -0.60
10. moonmoon: -0.54

--- Top 10 Positive Streamers: ---
1. realzbluewater: 29.88
2. lilsimsie: 19.80
3. justinbieber: 15.48
4. anthonydaytv: 15.32
5. dezgamez: 15.04
6. austinmcbroom: 15.00
7. abdulhd: 14.74
8. casinodaddy: 14.52
9. ferretsoftware: 13.69
10. commanderx: 13.22
